In [1]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from shutil import copyfile
from os import getcwd
import numpy as np

from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

In [2]:
#define our RCCNet model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 64)        2

In [3]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=6, min_lr=0.5e-15)
csv_logger = CSVLogger('RCCNet_F1score_NoDA_dummy.csv', append=True, separator=';')

In [4]:
adam = Adam(lr=0.00006, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1.0e-6,amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[tf.keras.metrics.CategoricalAccuracy(),
                           tfa.metrics.F1Score(num_classes=4, average='macro')])

/home/trungngo/Desktop/venv/colon-cancer-classification/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [5]:
TRAIN_DIR = '../../../data/cell_type/training'
TEST_DIR = '../../../data/cell_type/testing'
VAL_DIR = '../../../data/cell_type/validation'
INPUT_WIDTH = 32
INPUT_HEIGHT = 32

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=64
)



Found 7915 images belonging to 4 classes.
Found 990 images belonging to 4 classes.
Found 991 files belonging to 4 classes.


In [10]:
test_dataset = image_dataset_from_directory(TEST_DIR,
                                                  shuffle=True,
                                                  batch_size=64,
                                                  image_size=(32,32),
                                                  interpolation='nearest',
                                                  label_mode='categorical'
                                            )

Found 991 files belonging to 4 classes.


In [6]:
#Early stopping when training_acc > 0.99
# class myCallBack(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if logs.get('accuracy') >= 0.99:
#             print('\nReached 99% accuracy so cancelling training!')
#             self.model.stop_training = True

history = model.fit(train_generator, 
                    epochs=500, 
                    #steps_per_epoch=(total_training_imgs // 128), 
                    validation_data = validation_generator, 
                    verbose = 1,
#                     callbacks=[myCallBack()]
                    #validation_steps=(total_val_imgs // 64)
)

Epoch 1/500
124/124 [==============================] - ETA: 0s - loss: 1.2566 - categorical_accuracy: 0.5478 - f1_score: 0.4867

/home/trungngo/Desktop/venv/colon-cancer-classification/lib/python3.8/site-packages/tensorflow/python/keras/metrics.py:254: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  warnings.warn('Metric %s implements a `reset_states()` method; rename it '


124/124 [==============================] - 7s 32ms/step - loss: 1.2566 - categorical_accuracy: 0.5478 - f1_score: 0.4867 - val_loss: 2.0559 - val_categorical_accuracy: 0.4121 - val_f1_score: 0.1459
Epoch 2/500
124/124 [==============================] - 4s 30ms/step - loss: 1.0436 - categorical_accuracy: 0.6206 - f1_score: 0.5478 - val_loss: 3.4587 - val_categorical_accuracy: 0.4121 - val_f1_score: 0.1459
Epoch 3/500
124/124 [==============================] - 4s 30ms/step - loss: 0.9412 - categorical_accuracy: 0.6500 - f1_score: 0.5778 - val_loss: 3.0080 - val_categorical_accuracy: 0.4121 - val_f1_score: 0.1459
Epoch 4/500
124/124 [==============================] - 4s 30ms/step - loss: 0.9021 - categorical_accuracy: 0.6681 - f1_score: 0.5987 - val_loss: 1.5839 - val_categorical_accuracy: 0.4788 - val_f1_score: 0.2836
Epoch 5/500
124/124 [==============================] - 4s 30ms/step - loss: 0.8714 - categorical_accuracy: 0.6742 - f1_score: 0.6027 - val_loss: 0.7387 - val_categorical_ac

Epoch 40/500
124/124 [==============================] - 4s 30ms/step - loss: 0.6028 - categorical_accuracy: 0.7726 - f1_score: 0.7247 - val_loss: 0.5610 - val_categorical_accuracy: 0.7707 - val_f1_score: 0.7288
Epoch 41/500
124/124 [==============================] - 4s 30ms/step - loss: 0.5851 - categorical_accuracy: 0.7773 - f1_score: 0.7276 - val_loss: 0.5483 - val_categorical_accuracy: 0.7778 - val_f1_score: 0.7348
Epoch 42/500
124/124 [==============================] - 4s 31ms/step - loss: 0.5916 - categorical_accuracy: 0.7723 - f1_score: 0.7226 - val_loss: 0.5620 - val_categorical_accuracy: 0.7818 - val_f1_score: 0.7360
Epoch 43/500
124/124 [==============================] - 4s 31ms/step - loss: 0.5845 - categorical_accuracy: 0.7742 - f1_score: 0.7228 - val_loss: 0.5322 - val_categorical_accuracy: 0.7869 - val_f1_score: 0.7453
Epoch 44/500
124/124 [==============================] - 4s 30ms/step - loss: 0.5811 - categorical_accuracy: 0.7760 - f1_score: 0.7287 - val_loss: 0.5943 - v

Epoch 79/500
124/124 [==============================] - 4s 30ms/step - loss: 0.5096 - categorical_accuracy: 0.8072 - f1_score: 0.7657 - val_loss: 0.5096 - val_categorical_accuracy: 0.8051 - val_f1_score: 0.7680
Epoch 80/500
124/124 [==============================] - 4s 31ms/step - loss: 0.5047 - categorical_accuracy: 0.8064 - f1_score: 0.7673 - val_loss: 0.4906 - val_categorical_accuracy: 0.8162 - val_f1_score: 0.7712
Epoch 81/500
124/124 [==============================] - 4s 31ms/step - loss: 0.5019 - categorical_accuracy: 0.8100 - f1_score: 0.7703 - val_loss: 0.4583 - val_categorical_accuracy: 0.8333 - val_f1_score: 0.7929
Epoch 82/500
124/124 [==============================] - 4s 31ms/step - loss: 0.5007 - categorical_accuracy: 0.8092 - f1_score: 0.7683 - val_loss: 0.5379 - val_categorical_accuracy: 0.7929 - val_f1_score: 0.7521
Epoch 83/500
124/124 [==============================] - 4s 31ms/step - loss: 0.4910 - categorical_accuracy: 0.8112 - f1_score: 0.7693 - val_loss: 0.4606 - v

...

Epoch 157/500
124/124 [==============================] - 4s 30ms/step - loss: 0.4323 - categorical_accuracy: 0.8411 - f1_score: 0.8087 - val_loss: 0.4399 - val_categorical_accuracy: 0.8364 - val_f1_score: 0.7997
Epoch 158/500
124/124 [==============================] - 4s 30ms/step - loss: 0.4203 - categorical_accuracy: 0.8397 - f1_score: 0.8067 - val_loss: 0.5327 - val_categorical_accuracy: 0.8010 - val_f1_score: 0.7442
Epoch 159/500
124/124 [==============================] - 4s 30ms/step - loss: 0.4243 - categorical_accuracy: 0.8389 - f1_score: 0.8067 - val_loss: 0.4423 - val_categorical_accuracy: 0.8354 - val_f1_score: 0.7979
Epoch 160/500
124/124 [==============================] - 4s 30ms/step - loss: 0.4114 - categorical_accuracy: 0.8460 - f1_score: 0.8153 - val_loss: 0.4167 - val_categorical_accuracy: 0.8545 - val_f1_score: 0.8198
Epoch 161/500
124/124 [==============================] - 4s 30ms/step - loss: 0.4055 - categorical_accuracy: 0.8485 - f1_score: 0.8170 - val_loss: 0.460

Epoch 196/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3857 - categorical_accuracy: 0.8542 - f1_score: 0.8243 - val_loss: 0.4376 - val_categorical_accuracy: 0.8364 - val_f1_score: 0.8041
Epoch 197/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3938 - categorical_accuracy: 0.8524 - f1_score: 0.8226 - val_loss: 0.4792 - val_categorical_accuracy: 0.8162 - val_f1_score: 0.7836
Epoch 198/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3861 - categorical_accuracy: 0.8539 - f1_score: 0.8247 - val_loss: 0.5467 - val_categorical_accuracy: 0.7899 - val_f1_score: 0.7475
Epoch 199/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3917 - categorical_accuracy: 0.8500 - f1_score: 0.8200 - val_loss: 0.5463 - val_categorical_accuracy: 0.8091 - val_f1_score: 0.7731
Epoch 200/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3945 - categorical_accuracy: 0.8517 - f1_score: 0.8232 - val_loss: 0.499

Epoch 235/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3636 - categorical_accuracy: 0.8641 - f1_score: 0.8369 - val_loss: 0.4515 - val_categorical_accuracy: 0.8455 - val_f1_score: 0.8075
Epoch 236/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3649 - categorical_accuracy: 0.8606 - f1_score: 0.8338 - val_loss: 0.4728 - val_categorical_accuracy: 0.8354 - val_f1_score: 0.7941
Epoch 237/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3738 - categorical_accuracy: 0.8593 - f1_score: 0.8317 - val_loss: 0.4851 - val_categorical_accuracy: 0.8384 - val_f1_score: 0.8029
Epoch 238/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3709 - categorical_accuracy: 0.8599 - f1_score: 0.8331 - val_loss: 0.4806 - val_categorical_accuracy: 0.8374 - val_f1_score: 0.8028
Epoch 239/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3645 - categorical_accuracy: 0.8642 - f1_score: 0.8374 - val_loss: 0.457

Epoch 274/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3460 - categorical_accuracy: 0.8687 - f1_score: 0.8412 - val_loss: 0.4707 - val_categorical_accuracy: 0.8455 - val_f1_score: 0.8080
Epoch 275/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3410 - categorical_accuracy: 0.8695 - f1_score: 0.8444 - val_loss: 0.4753 - val_categorical_accuracy: 0.8343 - val_f1_score: 0.7974
Epoch 276/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3435 - categorical_accuracy: 0.8689 - f1_score: 0.8427 - val_loss: 0.4768 - val_categorical_accuracy: 0.8293 - val_f1_score: 0.7906
Epoch 277/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3414 - categorical_accuracy: 0.8709 - f1_score: 0.8452 - val_loss: 0.5301 - val_categorical_accuracy: 0.8232 - val_f1_score: 0.7821
Epoch 278/500
124/124 [==============================] - 4s 30ms/step - loss: 0.3414 - categorical_accuracy: 0.8752 - f1_score: 0.8496 - val_loss: 0.488

KeyboardInterrupt: 

In [11]:
loss, accuracy, f1_score = model.evaluate(test_dataset)

16/16 [==============================] - 0s 4ms/step - loss: 9924.9297 - categorical_accuracy: 0.2573 - f1_score: 0.1023


In [12]:
print(loss, accuracy, f1_score)

9924.9296875 0.25731584429740906 0.10232744365930557
